In [36]:
import os
import numpy as np
import cv2
import albumentations as A
from tensorflow.keras.preprocessing.image import img_to_array, load_img

In [37]:
input_dir = "dataset/"  # original image location
output_dir = "augmented/"  # augmented image location

In [38]:
# Enter class names
classes = ["Immature_Dragon_Fruit", "Mature_Dragon_Fruit"]
# Number of augmented images to generate per original image
num_augmented_images = 3
image_size = (300, 300)  # you can change image size

In [39]:
# Create output directories
for cls in classes:
    os.makedirs(os.path.join(output_dir, f'{cls}_aug'), exist_ok=True)

In [40]:
# Define an augmentation pipeline with recommended parameters
augmentation_pipeline = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.Transpose(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=45, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
    A.RGBShift(r_shift_limit=10, g_shift_limit=10, b_shift_limit=10, p=0.5),
    A.RandomGamma(gamma_limit=(80, 120), p=0.5),
    A.Blur(blur_limit=3, p=0.2),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.2),
    A.Resize(height=image_size[0], width=image_size[1])
])

In [41]:
# Function to load image
def load_image(image_path):
    return cv2.imread(image_path)

In [42]:
# Function to save image
def save_image(image_path, image):
    cv2.imwrite(image_path, image)

In [43]:
# Function to resize image
def resize_image(image, size):
    return cv2.resize(image, size)

In [44]:
# Count the number of images before augmentation and estimate the number of augmented images
print("Number of images before augmentation and estimated augmented images:")
total_original_images = 0
total_estimated_augmented_images = 0

for cls in classes:
    class_input_dir = os.path.join(input_dir, cls)
    num_images = len(os.listdir(class_input_dir))
    total_original_images += num_images
    estimated_augmented_images = num_images * num_augmented_images
    total_estimated_augmented_images += estimated_augmented_images
    print(f"{cls}: {num_images} images,\n estimated {cls}_augmented: {estimated_augmented_images} images\n")

print(f"Total original images: {total_original_images}\n")
print(f"Total estimated augmented images: {total_estimated_augmented_images}\n")

Number of images before augmentation and estimated augmented images:
Immature_Dragon_Fruit: 1241 images,
 estimated Immature_Dragon_Fruit_augmented: 3723 images

Mature_Dragon_Fruit: 886 images,
 estimated Mature_Dragon_Fruit_augmented: 2658 images

Total original images: 2127

Total estimated augmented images: 6381



In [45]:
# Iterate over each class and augment the images
cnt = 1

for cls in classes:
    class_input_dir = os.path.join(input_dir, cls)
    class_output_dir = os.path.join(output_dir, f'{cls}_aug')

    for img_name in os.listdir(class_input_dir):
        img_path = os.path.join(class_input_dir, img_name)
        image = load_image(img_path)
        resized_image = resize_image(image, image_size)
        
        for i in range(num_augmented_images):
            augmented_image = augmentation_pipeline(image=resized_image)['image']
            augmented_image_path = os.path.join(class_output_dir, f"{os.path.splitext(img_name)[0]}_aug_{cnt}.png")
            save_image(augmented_image_path, augmented_image)
            print(f"\rAugmented: {cnt}", end='', flush=True)
            cnt += 1

Augmented: 6381

In [46]:
# Count the number of images after augmentation
total_augmented_images = 0

for cls in classes:
    class_output_dir = os.path.join(output_dir, f"{cls}_aug")
    num_images = len(os.listdir(class_output_dir))
    total_augmented_images += num_images
    print(f"{cls}_augmented: {num_images} images")

print(f"\nTotal Augmented images: {total_augmented_images}")
print("Data Augmentation complete")

Immature_Dragon_Fruit_augmented: 3723 images
Mature_Dragon_Fruit_augmented: 2658 images

Total Augmented images: 6381
Data Augmentation complete
